# Salamandra 7b R1

En este notebook, utilizaremos el trabajo realizado por Unsloth para recorrer una muestra a pequeña escala del proceso DeepSeek-R1 descrito en su artículo.

Este no es un reflejo exacto 1 a 1, pero sí destaca las principales innovaciones presentadas en el artículo.

¡Vamos a ello!

### ¿Qué es el proceso de entrenamiento GRPO con RL?

1. **Group Sampling:** Para un solo prompt o estado, la política genera un lote de respuestas en lugar de solo una. Esto produce un pequeño “grupo” de posibles acciones o respuestas.

2. **Reward Scoring:** Cada respuesta se puntúa mediante una función de recompensa, que refleja qué tan buena o deseable es esa respuesta para la tarea en cuestión.

3. **Group-Based Advantage:** El algoritmo calcula la “ventaja” de cada respuesta comparando su recompensa con la recompensa promedio del grupo. Si la recompensa de una respuesta está por encima del promedio, tiene una ventaja positiva (y viceversa).

4. **Policy Update:** La política se ajusta para promover las respuestas con ventaja positiva y desalentar aquellas con ventaja negativa. Se incluye un término de penalización KL para evitar cambios demasiado drásticos en la política.

5. **Proceso iterativo:** La política actualizada se usa nuevamente para generar nuevos grupos, evaluarlos y actualizarse, repitiendo el proceso hasta que la política converja o alcance los objetivos de rendimiento.

Este enfoque basado en grupos elimina la necesidad de una función de valor separada (crítico) y ayuda a la política a aprender rápidamente qué respuestas son relativamente mejores dentro de cada grupo muestreado.

> **NOTA:** Este notebook se basa en gran medida en los notebooks del modelo


# Unsloth AI: Plataforma de Fine-Tuning para Modelos de Lenguaje

Unsloth AI es una plataforma diseñada para agilizar y acelerar el finetuning de grandes modelos de lenguaje (LLMs) como Llama, Mistral y Gemma2. Ayuda a desarrolladores y organizaciones a optimizar estos modelos para tareas o dominios específicos de manera eficiente. Unsloth es de código abierto y destaca por su facilidad de uso, permitiendo ajustar modelos con un rendimiento mejorado.

## Aspectos clave de Unsloth AI:

### 🚀 Eficiencia
Unsloth puede hacer que el proceso de finetuning sea **2 veces más rápido** que los métodos tradicionales, utilizando **un 70% menos de memoria sin perder precisión**. Optimiza los pasos matemáticos más intensivos en cómputo y reescribe kernels de GPU para lograr un entrenamiento más rápido sin necesidad de cambiar el hardware.

### 🔄 Versatilidad
Soporta una amplia variedad de LLMs, incluyendo **Llama (versiones 1, 2 y 3), Mistral, Gemma y Phi-3**. Puede ajustar modelos para tareas como **aprendizaje de idiomas, resumen de texto, IA conversacional y seguimiento de instrucciones**.

### 📈 Escalabilidad
Unsloth es compatible con configuraciones que van desde **una sola GPU hasta entornos con 8 GPUs e incluso configuraciones multi-nodo**, lo que lo hace adecuado tanto para proyectos pequeños como de gran escala.

### 🛠️ Facilidad de uso
La plataforma está diseñada para ser intuitiva y fácil de instalar, con **documentación completa y guías paso a paso**. Se puede configurar localmente o integrarse con plataformas como **Google Colab**.

### 🔗 Integración
Unsloth permite una integración fluida con herramientas de terceros como **Google Colab**, mejorando su funcionalidad y utilidad para el desarrollo avanzado de IA.

Unsloth es reconocido como una de las principales herramientas de código abierto para


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

### Instal dependencies

In [ ]:
%%capture
# Skip restarting message in Colab
import sys; modules = list(sys.modules.keys())
for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None

!pip install unsloth vllm
!pip install --upgrade pillow
# If you are running this notebook on local, you need to install `diffusers` too
# !pip install diffusers
# Temporarily install a specific TRL nightly version
!pip install git+https://github.com/huggingface/trl.git@e95f9fb74a3c3647b86f251b7e230ec51c64b72b

### Unsloth

Utiliza [`PatchFastRL`](https://github.com/unslothai/unsloth/blob/646ad2f141a3a0721d1ec9449cf9454b5612a84a/unsloth/models/rl.py#L44) antes de todas las funciones para aplicar parches a **GRPO** y otros algoritmos de **Refuerzo** (RL).

> **NOTA:** Este parche sobrescribe `.generate` de **TRL** para hacerlo más optimizado. ¡Típico de Unsloth!


In [ ]:
from unsloth import FastLanguageModel, PatchFastRL
PatchFastRL("GRPO", FastLanguageModel)

Unsloth: Patching Xformers to fix some performance issues.
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 02-18 10:13:44 __init__.py:190] Automatically detected platform cuda.


Esta celda se encargará de cargar nuestro modelo y de establecer algunos hiperparámetros específicos de **LoRA**, como es habitual.

Hay mucho ocurriendo en segundo plano, pero la idea es simple: **Unsloth está aplicando parches para optimizar el proceso**.

> **NOTA:** Puedes echar un vistazo al [blog de Unsloth](https://unsloth.ai/blog/r1-reasoning) si quieres más información sobre cómo entrenar estos modelos de la mejor manera.


In [ ]:
from unsloth import is_bfloat16_supported
import torch

max_seq_length = 1024  # Puede aumentarse para rastrear razonamientos más largos
lora_rank = 32  # Un rango mayor = modelo más inteligente, pero más lento

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="BSC-LT/salamandra-7b-instruct",
    max_seq_length=max_seq_length,
    load_in_4bit=False,  # False para LoRA en 16 bits
    fast_inference=True,  # Habilitar inferencia rápida con vLLM
    max_lora_rank=lora_rank,
    gpu_memory_utilization=0.6,  # Reducir si hay problemas de memoria
)

model = FastLanguageModel.get_peft_model(
    model,
    r=lora_rank,  # Se puede elegir cualquier número > 0. Recomendados: 8, 16, 32, 64, 128
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],  # Eliminar QKVO si hay problemas de memoria
    lora_alpha=lora_rank,
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
)


==((====))==  Unsloth 2025.2.12: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading BSC-LT/salamandra-7b-instruct with actual GPU utilization = 59.37%
Unsloth: Your GPU has CUDA compute capability 8.0 with VRAM = 39.56 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 1024. Num Sequences = 224.
Unsloth: vLLM's KV Cache can use up to 8.88 GB. Also swap space = 6 GB.
INFO 02-18 10:14:03 config.py:542] This model supports multiple tasks: {'reward', 'classify', 'score', 'embed', 'generate'}. Defaulting to 'generate'.
INFO 02-18 10:14:03 llm_engine.py:234] Initializing a V0 L

tokenizer_config.json:   0%|          | 0.00/3.80k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.81M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/19.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/513 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/3.80k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.81M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/513 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/19.1M [00:00<?, ?B/s]

Unsloth: Will load BSC-LT/salamandra-7b-instruct as a legacy tokenizer.
Unsloth 2025.2.12 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


### Test del modelo
Vamos a testear el modelo antes de hacer nada, para ver que tal se desenvuelve

In [ ]:
problemas = [
    "La Laura vol comprar llaminadures per a una festa. Cada bossa de llaminadures costa 2,50€ i ella vol comprar 8 bosses. A més, veu que hi ha una oferta especial: si compra 10 bosses, obté un descompte del 20% en el preu total. Quant haurà de pagar si compra només les 8 bosses? Quant li costaria si comprés 10 bosses amb el descompte aplicat? És millor comprar 8 bosses o aprofitar l'oferta i comprar-ne 10? Quant estalviaria o gastaria de més?",

    "L’avi de l’Arnau té un hort amb pomeres i peres. En total, té 45 arbres fruiters. Si el nombre de pomeres és el doble que el de pereres, quants arbres de cada tipus té?",

    "Un ciclista recorre 60 km en 3 hores a velocidad constante. Si mantiene la misma velocidad, ¿cuántos kilómetros recorrerá en 5 horas?",

    "Andrea tiene 50 euros y quiere comprar 3 libros. Cada libro cuesta 12 euros. ¿Le alcanza el dinero para comprarlos? Si no, ¿cuánto dinero le falta?"
]

In [ ]:
from vllm import SamplingParams

sampling_params = SamplingParams(
    temperature=0.6,
    top_p=0.95,
    max_tokens=1024,
)

for problema in problemas:
    text = tokenizer.apply_chat_template([
        {"role": "user", "content": problema},
    ], tokenize=False, add_generation_prompt=True)

    output = model.fast_generate(
        [text],
        sampling_params=sampling_params,
        lora_request=None,
    )[0].outputs[0].text

    print(f"🔹 **Problema:** {problema}\n🔹 **Respuesta:** {output}\n{'-'*50}\n")

Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  1.03it/s, est. speed input: 454.84 toks/s, output: 63.06 toks/s]


🔹 **Problema:** La Laura vol comprar llaminadures per a una festa. Cada bossa de llaminadures costa 2,50€ i ella vol comprar 8 bosses. A més, veu que hi ha una oferta especial: si compra 10 bosses, obté un descompte del 20% en el preu total. Quant haurà de pagar si compra només les 8 bosses? Quant li costaria si comprés 10 bosses amb el descompte aplicat? És millor comprar 8 bosses o aprofitar l'oferta i comprar-ne 10? Quant estalviaria o gastaria de més?
🔹 **Respuesta:** La Laura haurà de pagar 16 euros si compra només les 8 bosses. Si compra 10 bosses amb el descompte aplicat, haurà de pagar 12,80 euros. És millor comprar 8 bosses perquè estalviaria 3,20 euros.
--------------------------------------------------



Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  1.47it/s, est. speed input: 529.62 toks/s, output: 67.86 toks/s]


🔹 **Problema:** L’avi de l’Arnau té un hort amb pomeres i peres. En total, té 45 arbres fruiters. Si el nombre de pomeres és el doble que el de pereres, quants arbres de cada tipus té?
🔹 **Respuesta:** L’avi d’en Arnau té 45 arbres fruiters. Si el nombre de pomeres és el doble que el de pereres, en té 9 pomeres i 4,5 pereres.
--------------------------------------------------



Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  6.20it/s, est. speed input: 2137.06 toks/s, output: 56.23 toks/s]


🔹 **Problema:** Un ciclista recorre 60 km en 3 hores a velocidad constante. Si mantiene la misma velocidad, ¿cuántos kilómetros recorrerá en 5 horas?
🔹 **Respuesta:** Recorrerá 90 kilómetros.
--------------------------------------------------



Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  3.18it/s, est. speed input: 1119.70 toks/s, output: 63.98 toks/s]

🔹 **Problema:** Andrea tiene 50 euros y quiere comprar 3 libros. Cada libro cuesta 12 euros. ¿Le alcanza el dinero para comprarlos? Si no, ¿cuánto dinero le falta?
🔹 **Respuesta:** Sí, Andrea tiene suficiente dinero para comprar los 3 libros. Le faltan 3 euros.
--------------------------------------------------



✅ Conclusión: El modelo cometió errores graves en todos los problemas, por lo que no es fiable para resolver matemáticas sin verificación.

### Preparación de Datos

Aquí notarás algo peculiar: ¡nuestro dataset solo contiene entradas y salidas! (específicamente del conjunto de datos **GSM8K**).

Pero espera, dijimos que esto era diferente de **SFT**… ¡y sin embargo, parece exactamente lo mismo!

Bueno, todavía necesitamos preguntas y respuestas para verificar que estamos aprendiendo *algo* útil. Sin embargo, lo importante es que **no estamos utilizando un modelo de recompensa basado en la preferencia humana ni un modelo de recompensa de proceso** para incrustar respuestas en el modelo. Solo necesitamos una forma de verificar si una respuesta generada por el modelo es correcta o incorrecta. Es decir, ¡una manera de *recompensar* las respuestas correctas!

Por ahora, veamos cómo luce nuestro conjunto de datos de entrada.

> **NOTA:** Unsloth aprovechó directamente el trabajo realizado por [@willccbb](https://gist.github.com/willccbb/4676755236bb08cab5f4e54a0475d6fb) para la preparación de datos y todas las funciones de recompensa.


In [ ]:
import re
from datasets import load_dataset, concatenate_datasets, Dataset

# Carga y prepara el dataset
SYSTEM_PROMPT = """
Respond in the following format:
<reasoning>
...
</reasoning>
<answer>
...
</answer>
"""

XML_COT_FORMAT = """\
<reasoning>
{reasoning}
</reasoning>
<answer>
{answer}
</answer>
"""

def extract_xml_answer(text: str) -> str:
    answer = text.split("<answer>")[-1]
    answer = answer.split("</answer>")[0]
    return answer.strip()

def extract_hash_answer(text: str) -> str | None:
    if "####" not in text:
        return None
    return text.split("####")[1].strip()

def load_and_format_dataset(dataset_name: str, config: str = "", split="train") -> Dataset:
    data = load_dataset(dataset_name, config)[split]  # type: ignore
    data = data.map(lambda x: {  # type: ignore
        'prompt': [
            {'role': 'system', 'content': SYSTEM_PROMPT},
            {'role': 'user', 'content': x['question']}
        ],
        'answer': extract_hash_answer(x['answer'])
    })  # type: ignore
    return data  # type: ignore

# Cargar los tres datasets
datasets = [
    load_and_format_dataset('openai/gsm8k', 'main'),
    load_and_format_dataset('ericrisco/gsm8k-translated-spanish'),
    load_and_format_dataset('ericrisco/gsm8k-translated-catalan'),
]

# Fusionar los datasets
dataset = concatenate_datasets(datasets)

# Mostrar el tamaño final del dataset combinado
print(f"Tamaño del dataset combinado: {len(dataset)}")


README.md:   0%|          | 0.00/7.94k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/2.31M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/419k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7473 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1319 [00:00<?, ? examples/s]

...

test-00000-of-00001.parquet:   0%|          | 0.00/442k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7473 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1319 [00:00<?, ? examples/s]

Map:   0%|          | 0/7473 [00:00<?, ? examples/s]

Tamaño del dataset combinado: 22419


Como puedes ver en estos datos, no hay información específica sobre preferencias, "cómo razonar" o algo similar. Simplemente se presentan la pregunta y la respuesta.

Esta es la idea central detrás de este estilo de entrenamiento: **no le diremos al modelo *cómo* debe pensar**, simplemente lo dejaremos jugar en un entorno definido por la pregunta y la respuesta.

> **NOTA:** Esto no es el caso de **DeepSeek-R1**, donde hay una **pequeña** cantidad de **SFT** (conocida como "cold-start") para "preparar" el modelo antes de la fase de entrenamiento con **RL**.


In [ ]:
dataset[0]

{'question': 'Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?',
 'answer': '72',
 'prompt': [{'content': '\nRespond in the following format:\n<reasoning>\n...\n</reasoning>\n<answer>\n...\n</answer>\n',
   'role': 'system'},
  {'content': 'Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?',
   'role': 'user'}]}

Ahora llegamos a la *magia* de este enfoque: una colección de **modelos de recompensa**.

Observa que realizamos una serie de **"verificaciones"**, las cuales se combinan como se muestra en el siguiente diagrama:

![image](https://i.imgur.com/7Dp0qdt.png)

Básicamente, esto significa que utilizamos un conjunto de **funciones de recompensa** para evaluar si nuestro modelo está aprendiendo *como queremos*, en lugar de proporcionarle ejemplos explícitos que le indiquen *cómo* debería aprender.

Estas funciones de recompensa son completamente **personalizables**, lo que permite a los usuarios **dirigir** eficazmente **cómo** y **en qué** debe especializarse el modelo.


In [ ]:
# Reward functions
def correctness_reward_func(prompts, completions, answer, **kwargs) -> list[float]:
    responses = [completion[0]['content'] for completion in completions]
    q = prompts[0][-1]['content']
    extracted_responses = [extract_xml_answer(r) for r in responses]
    print('-'*20, f"Question:\n{q}", f"\nAnswer:\n{answer[0]}", f"\nResponse:\n{responses[0]}", f"\nExtracted:\n{extracted_responses[0]}")
    return [2.0 if r == a else 0.0 for r, a in zip(extracted_responses, answer)]

def int_reward_func(completions, **kwargs) -> list[float]:
    responses = [completion[0]['content'] for completion in completions]
    extracted_responses = [extract_xml_answer(r) for r in responses]
    return [0.5 if r.isdigit() else 0.0 for r in extracted_responses]

def strict_format_reward_func(completions, **kwargs) -> list[float]:
    """Reward function that checks if the completion has a specific format."""
    pattern = r"^<reasoning>\n.*?\n</reasoning>\n<answer>\n.*?\n</answer>\n$"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]

def soft_format_reward_func(completions, **kwargs) -> list[float]:
    """Reward function that checks if the completion has a specific format."""
    pattern = r"<reasoning>.*?</reasoning>\s*<answer>.*?</answer>"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]

def count_xml(text) -> float:
    count = 0.0
    if text.count("<reasoning>\n") == 1:
        count += 0.125
    if text.count("\n</reasoning>\n") == 1:
        count += 0.125
    if text.count("\n<answer>\n") == 1:
        count += 0.125
        count -= len(text.split("\n</answer>\n")[-1])*0.001
    if text.count("\n</answer>") == 1:
        count += 0.125
        count -= (len(text.split("\n</answer>")[-1]) - 1)*0.001
    return count

def xmlcount_reward_func(completions, **kwargs) -> list[float]:
    contents = [completion[0]["content"] for completion in completions]
    return [count_xml(c) for c in contents]

### Entrenar el modelo

Ahora que tenemos:

1. **Ejemplos de entrenamiento**  
2. **Funciones de recompensa**  

¡Solo nos queda entrenar nuestro modelo!

Comenzaremos configurando una serie de **hiperparámetros**.

> **NOTA:** Estos hiperparámetros están optimizados para la instancia gratuita de **Colab T4**, pero puedes modificarlos para adaptarlos mejor a tu hardware.


### GPROConfig

Lo primero y más importante: tenemos una serie de **hiperparámetros típicos** (como siempre).

También notarás una notable *ausencia* de hiperparámetros específicos de **GRPO** en esta implementación. Nos quedaremos con los valores por defecto para mantener este notebook manejable, pero eres libre de explorar **TRL** y ajustar los parámetros para encontrar la mejor configuración según tu caso de uso.

> **NOTA:** Si quieres obtener la clásica imagen de **RL** con la "línea subiendo hacia la derecha", puedes eliminar `report_to = "none"` de la siguiente configuración.


In [ ]:
from trl import GRPOConfig, GRPOTrainer

training_args = GRPOConfig(
    use_vllm=True,  # Usar vLLM para inferencia rápida
    learning_rate=5e-6,
    adam_beta1=0.9,
    adam_beta2=0.99,
    weight_decay=0.1,
    warmup_ratio=0.1,
    lr_scheduler_type="cosine",
    optim="paged_adamw_8bit",
    logging_steps=1,
    bf16=is_bfloat16_supported(),  # Usar bf16 si es compatible con el hardware
    fp16=not is_bfloat16_supported(),  # Usar fp16 si bf16 no es compatible
    per_device_train_batch_size=1,  # Tamaño de batch por dispositivo
    gradient_accumulation_steps=1,  # Aumentar a 4 para un entrenamiento más estable
    num_generations=6,  # Reducir si hay problemas de memoria
    max_prompt_length=256,  # Longitud máxima del prompt
    max_completion_length=200,  # Longitud máxima de la respuesta generada
    #num_train_epochs = 1, # Establecer en 1 para una ejecución de entrenamiento completa
    max_steps=500,  # Número máximo de pasos de entrenamiento
    save_steps=500,  # Guardar el modelo cada 500 pasos
    max_grad_norm=0.1,  # Normalización del gradiente
    report_to="none",  # Se puede cambiar a Weights & Biases para registrar métricas
    output_dir="outputs",  # Directorio de salida para guardar el modelo
)


¡Finalmente, podemos ejecutar nuestro **trainer**!

La idea principal en este enfoque basado en **RL** es que, en lugar de observar cómo disminuye la pérdida (*loss*), queremos ver cómo la **recompensa aumenta**.

> **NOTA:** El entrenamiento tiene un momento de "¡Aha!", como se ha descrito, en el que la recompensa pasa de ~0 y, de repente, comienza a aumentar. Este comportamiento es esperado, pero es posible que **no veas cambios en la columna de recompensa** (que refleja la salida combinada de nuestras funciones de recompensa definidas anteriormente) hasta después de los pasos **100-150**.


In [ ]:
trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        xmlcount_reward_func,
        soft_format_reward_func,
        strict_format_reward_func,
        int_reward_func,
        correctness_reward_func,
    ],
    args = training_args,
    train_dataset = dataset,
)
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 22,419 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 1 | Gradient Accumulation steps = 1
\        /    Total batch size = 1 | Total steps = 500
 "-____-"     Number of trainable parameters = 73,662,464


-------------------- Question:
Un tiquet de concert costa 40 $. El senyor Benson va comprar 12 tiquets i va rebre un descompte del 5% per cada tiquet comprat que supera 10. Quant va pagar en total el senyor Benson? 
Answer:
476 
Response:
En total, el senyor Benson va pagar 476 $ per tots els tiquets de concert. 
Extracted:
En total, el senyor Benson va pagar 476 $ per tots els tiquets de concert.


Step,Training Loss,reward,reward_std,completion_length,kl,rewards / xmlcount_reward_func,rewards / soft_format_reward_func,rewards / strict_format_reward_func,rewards / int_reward_func,rewards / correctness_reward_func
1,0.000000,0.000000,0.000000,37.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,200.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,104.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,119.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.000000,0.000000,0.000000,189.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.000000,0.000000,0.000000,82.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,0.000000,0.000000,0.000000,96.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.000000,0.000000,0.000000,200.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.000000,0.000000,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
10,0.000000,0.000000,0.000000,161.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


Se han truncado las últimas 5000 líneas del flujo de salida.
One month later, the song has 120,000 listens.

Two months later, the song has 240,000 listens.

Three months later, the song has 480,000 list 
Extracted:
Jordan's song has 60,000 listens. However, this won't be the total number of listens for the year. It will be her inital starting point for the song's popularity trend for the year.

Each month, the listen count doubles due to popularity and word of mouth. (This is also reflected by the 3 months left in the year, which implies we are 9 months into the year when we look at the start of the song.) 

So, the song has 60,000 listens at the start of the year. Every month after, her listen count doubles. 

One month later, the song has 120,000 listens.

Two months later, the song has 240,000 listens.

Three months later, the song has 480,000 list
-------------------- Question:
A local park has 70 pigeons that call the park home. Half of the pigeons are black, and 20 percent of th

TrainOutput(global_step=500, training_loss=5.578486914600944e-05, metrics={'train_runtime': 1676.6863, 'train_samples_per_second': 0.298, 'train_steps_per_second': 0.298, 'total_flos': 0.0, 'train_loss': 5.578486914600944e-05})

### Testear el modelo razonador
Ahora vamos a testear el modelo razonador y lo compararemos con el modelo sin razonador!

In [ ]:
problemas = [
    "La Laura vol comprar llaminadures per a una festa. Cada bossa de llaminadures costa 2,50€ i ella vol comprar 8 bosses. A més, veu que hi ha una oferta especial: si compra 10 bosses, obté un descompte del 20% en el preu total. Quant haurà de pagar si compra només les 8 bosses? Quant li costaria si comprés 10 bosses amb el descompte aplicat? És millor comprar 8 bosses o aprofitar l'oferta i comprar-ne 10? Quant estalviaria o gastaria de més?",

    "L’avi de l’Arnau té un hort amb pomeres i peres. En total, té 45 arbres fruiters. Si el nombre de pomeres és el doble que el de pereres, quants arbres de cada tipus té?",

    "Un ciclista recorre 60 km en 3 horas a velocidad constante. Si mantiene la misma velocidad, ¿cuántos kilómetros recorrerá en 5 horas?",

    "Andrea tiene 50 euros y quiere comprar 3 libros. Cada libro cuesta 12 euros. ¿Le alcanza el dinero para comprarlos? Si no, ¿cuánto dinero le falta?"
]

In [ ]:
from vllm import SamplingParams

SYSTEM_PROMPT = """
Respond in the following format:
<reasoning>
...
</reasoning>
<answer>
...
</answer>
"""

sampling_params = SamplingParams(
    temperature=0.6,
    top_p=0.95,
    max_tokens=1024,
)

model.save_lora("grpo_saved_lora")

In [ ]:
for problema in problemas:

    text = tokenizer.apply_chat_template([
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": problema},
    ], tokenize=False, add_generation_prompt=True)

    # Generar respuesta modelo base
    output_base = model.fast_generate(
        text,
        sampling_params=sampling_params,
        lora_request=None,
    )[0].outputs[0].text

    # Generar respuesta razonamiento activado
    output_lora = model.fast_generate(
        text,
        sampling_params=sampling_params,
        lora_request=model.load_lora("grpo_saved_lora"),
    )[0].outputs[0].text

    # Imprimir los resultados comparativos
    print(f"🔹 **Problema:** {problema}")
    print(f"🔹 **Modelo base:**\n{output_base}\n")
    print(f"🔹 **Modelo razonador:**\n{output_lora}\n")
    print("-" * 80)


Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.81s/it, est. speed input: 97.40 toks/s, output: 59.76 toks/s]


🔹 **Problema:** La Laura vol comprar llaminadures per a una festa. Cada bossa de llaminadures costa 2,50€ i ella vol comprar 8 bosses. A més, veu que hi ha una oferta especial: si compra 10 bosses, obté un descompte del 20% en el preu total. Quant haurà de pagar si compra només les 8 bosses? Quant li costaria si comprés 10 bosses amb el descompte aplicat? És millor comprar 8 bosses o aprofitar l'oferta i comprar-ne 10? Quant estalviaria o gastaria de més?
🔹 **Modelo base:**
Si Laura vol comprar 8 bosses de llaminadures, haurà de pagar 20 euros en total (2,50 € per bossa x 8 bosses).

Si Laura compra 10 bosses amb el descompte del 20%, el preu total serà de 16 euros (10 bosses x 2,50 € per bossa - 20%).

Per tant, Laura estalviaria 4 euros si compra 10 bosses en lloc de 8.

En termes generals, és millor comprar 8 bosses de llaminadures, ja que estalviaria 4 euros.

🔹 **Modelo razonador:**
Si Laura compra només 8 bosses de llaminadures, haurà de pagar 20 euros. Si aprofita l'oferta i com

Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.12s/it, est. speed input: 84.57 toks/s, output: 55.19 toks/s]


🔹 **Problema:** L’avi de l’Arnau té un hort amb pomeres i peres. En total, té 45 arbres fruiters. Si el nombre de pomeres és el doble que el de pereres, quants arbres de cada tipus té?
🔹 **Modelo base:**
El nombre de pomeres és el doble que el de pereres, per tant, hi ha 2x pereres = 2*15 = 30 pomeres. En total, l’avi de l’Arnau té 30 + 15 = 45 arbres fruiters.

🔹 **Modelo razonador:**
El nombre d’arbres de cada tipus és el mateix. Si l’avi té 45 arbres fruiters en total, i el nombre de pomeres és el doble que el de pereres, aleshores té 2*30 = 60 pomeres i 30 peres.

--------------------------------------------------------------------------------


Processed prompts: 100%|██████████| 1/1 [00:02<00:00,  2.81s/it, est. speed input: 27.77 toks/s, output: 62.66 toks/s]


🔹 **Problema:** Un ciclista recorre 60 km en 3 horas a velocidad constante. Si mantiene la misma velocidad, ¿cuántos kilómetros recorrerá en 5 horas?
🔹 **Modelo base:**
El ciclista recorrerá 90 km en 5 horas.

🔹 **Modelo razonador:**
Para resolver este problema, podemos usar la fórmula para calcular la distancia recorrida en un tiempo dado:

Distancia = Velocidad * Tiempo

En este caso, la velocidad es constante, por lo que podemos reescribir la fórmula como:

Distancia = Velocidad * Tiempo²

Sabemos que el ciclista recorre 60 km en 3 horas, por lo que podemos sustituir los valores en la fórmula y calcular la distancia recorrida en 5 horas:

Distancia = 60 km/3 h * 3 h²

Distancia = 60 km * 9 h²

Distancia = 540 km²

Después de simplificar, obtenemos que el ciclista recorrerá 54 km en 5 horas.

--------------------------------------------------------------------------------


Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  1.10it/s, est. speed input: 94.79 toks/s, output: 51.81 toks/s]

🔹 **Problema:** Andrea tiene 50 euros y quiere comprar 3 libros. Cada libro cuesta 12 euros. ¿Le alcanza el dinero para comprarlos? Si no, ¿cuánto dinero le falta?
🔹 **Modelo base:**
Si Andrea quiere comprar 3 libros de 12 euros cada uno, el total que tendrá que pagar será de 36 euros. Como Andrea tiene 50 euros, le faltarán 14 euros para poder comprar los libros.

🔹 **Modelo razonador:**
Si Andrea quiere comprar 3 libros de 12 euros cada uno, el total que tiene que pagar es 36 euros. Como Andrea tiene 50 euros, le faltan 14 euros para comprar los libros.

--------------------------------------------------------------------------------


# Subir versión a HF

In [ ]:
import torch

HF_USERNAME = "ericrisco"
MODEL_NAME = "salamandra-7b-r1"
HF_REPO_ID = f"{HF_USERNAME}/{MODEL_NAME}"

model.save_pretrained_merged(MODEL_NAME, tokenizer, save_method="merged_16bit")

model.push_to_hub_merged(HF_REPO_ID, tokenizer, save_method="merged_16bit")

model.save_pretrained_gguf(MODEL_NAME, tokenizer, quantization_method="f16")

model.push_to_hub_gguf(HF_REPO_ID, tokenizer, quantization_method="f16")

Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 15.6G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 46.55 out of 83.48 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


 53%|█████▎    | 17/32 [00:00<00:00, 59.04it/s]
We will save to Disk and not RAM now.
100%|██████████| 32/32 [00:08<00:00,  3.84it/s]


Unsloth: Saving tokenizer... Done.
Done.


Unsloth: You are pushing to hub, but you passed your HF username = ericrisco.
We shall truncate ericrisco/salamandra-7b-r1 to salamandra-7b-r1


...

==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits might take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['f16'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: CMAKE detected. Finalizing some steps for installation.
Unsloth: [1] Converting model at salamandra-7b-r1 into f16 GGUF format.
The output location will be /content/salamandra-7b-r1/unsloth.F16.gguf
This might take 3 minutes...
INFO:hf-to-gguf:Loading model: salamandra-7b-r1
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:gguf: loading model weight map from 'model.safetensors.index.json'
INFO:hf-to-gguf:gguf: loading model part 'model-00001-of-00004.safetensors'
INFO:hf-to-gguf:token_embd.weight,           torc

100%|██████████| 32/32 [00:00<00:00, 82.46it/s]


Unsloth: Saving tokenizer... Done.
Done.
==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits might take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['f16'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: [1] Converting model at ericrisco/salamandra-7b-r1 into f16 GGUF format.
The output location will be /content/ericrisco/salamandra-7b-r1/unsloth.F16.gguf
This might take 3 minutes...
INFO:hf-to-gguf:Loading model: salamandra-7b-r1
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:gguf: loading model weight map from 'model.safetensors.index.json'
INFO:hf-to-gguf:gguf: loading model part 'model-00001-of-00004.safetensors'
INFO:hf-to-gguf:token_embd.weight,           torch.bf

unsloth.F16.gguf:   0%|          | 0.00/15.5G [00:00<?, ?B/s]

Saved GGUF to https://huggingface.co/ericrisco/salamandra-7b-r1


# Testear el modelo sin entrenamiento

En esta parte descargamos el modelo de HF para testear tanto la versión R1 como la versión base del modelo para poder comparar con "problemas" que no ha visto. Se puede ejecutar desde aquí sin ejecutar todo lo anterior.

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

BASE_MODEL = "BSC-LT/salamandra-7b-instruct"
TUNED_MODEL = "ericrisco/salamandra-7b-r1"

base_model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, torch_dtype=torch.float16, device_map="auto")
tuned_model = AutoModelForCausalLM.from_pretrained(TUNED_MODEL, torch_dtype=torch.float16, device_map="auto")

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)

SYSTEM_PROMPT = """
Respond in the following format:
<reasoning>
...
</reasoning>
<answer>
...
</answer>
"""

In [ ]:
problemas = [
    "En una botiga venen caixes de galetes en promoció. Si cada caixa conté 24 galetes i compres 3 caixes, quantes galetes tindràs en total?",
    "En un autobús hi ha 35 passatgers. A la següent parada en baixen 12 i en pugen 7. Quants passatgers hi ha ara a l'autobús?",
    "Un granjero tiene 5 gallinas, cada una pone 4 huevos al día. ¿Cuántos huevos tendrá en una semana?",
    "Un coche recorre 450 km en 6 horas. Si mantiene la misma velocidad, ¿cuántos kilómetros recorrerá en 9 horas?"
]

In [ ]:
for problema in problemas:
    # Aplicar la plantilla de xat per generar el prompt correcte
    text = tokenizer.apply_chat_template([
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": problema},
    ], tokenize=False, add_generation_prompt=True)

    # Tokenitzar el prompt
    inputs = tokenizer(text, return_tensors="pt").to("cuda")

    # 🔹 Inferència amb el model base
    with torch.no_grad():
        output_base_tokens = base_model.generate(**inputs, max_new_tokens=200)
    output_base_text = tokenizer.decode(output_base_tokens[0], skip_special_tokens=True)

    # 🔹 Inferència amb el model ajustat
    with torch.no_grad():
        output_tuned_tokens = tuned_model.generate(**inputs, max_new_tokens=200)
    output_tuned_text = tokenizer.decode(output_tuned_tokens[0], skip_special_tokens=True)

    # 📌 Mostrar comparació de resultats
    print(f"🔹 **Problema:** {problema}\n")
    print(f"🟦 **Modelo Base ({BASE_MODEL}):**\n{output_base_text}\n")
    print(f"🟩 **Modelo Razonador ({TUNED_MODEL}):**\n{output_tuned_text}\n")
    print("=" * 100, "\n")
